In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.metrics import mean_squared_error
pd.set_option('mode.chained_assignment', None)

## Reading External Data

In [ ]:
gscpi_data = pd.read_excel("gscpi_data_clean.xlsx")
gscpi_data["Date"] = pd.to_datetime(gscpi_data["Date"])
gscpi_data.sort_values(by="Date")
gscpi_data["year"] = gscpi_data["Date"].dt.year
gscpi_data["month"] = gscpi_data["Date"].dt.month
#Shift by one and then join as gscpi is backward looking
gscpi_data = gscpi_data.set_index(["year","month"])["gscpi"].shift(1).reset_index()

consumer_sentiment_data = pd.read_excel("consumer_sentiment_clean.xlsx")
consumer_sentiment_data.rename(columns = {"Year":"year","Month_no":"month","Index":"consumer_sentiment_index"},inplace=True)
consumer_sentiment_data = consumer_sentiment_data[["year","month","consumer_sentiment_index"]]
consumer_sentiment_data.sort_values(by=["year","month"])
consumer_sentiment_data = consumer_sentiment_data.set_index(["year","month"])["consumer_sentiment_index"].shift(1).reset_index()


## Reading Train and Validation Data

In [ ]:
raw_data = pd.read_excel("Hackathon Data.xlsx")
validation_raw_data = pd.read_excel("Validation_Data.xlsx")
raw= raw_data.copy()
validation_raw_data = validation_raw_data.drop("Forecasted Units",axis=1)
raw_data = pd.concat([raw_data,validation_raw_data],axis=0)
master_sku_list = raw_data[["Encoded_SKU_ID"]].drop_duplicates().reset_index(drop=True)

raw_data["month"] = raw_data["SALES_DATE"].dt.month
raw_data["quarter"] = raw_data["SALES_DATE"].dt.quarter
raw_data["leap_year"] = raw_data["SALES_DATE"].dt.is_leap_year.astype(int)
raw_data["year"] = raw_data["SALES_DATE"].dt.year
raw_data["week"] = raw_data["SALES_DATE"].dt.isocalendar().week
raw_data["day_of_week"] = raw_data["SALES_DATE"].dt.day_of_week
raw_data = pd.get_dummies(raw_data,columns = ["Inventory"],drop_first=False)

# merge with external data
raw_data =raw_data.merge(consumer_sentiment_data,on=["year","month"],how="left")\
    .merge(gscpi_data,on=["year","month"],how="left")

## Creating data  - Feature Engineering

In [ ]:
def create_data_for_sku(sku_id):
    sku_1 = raw_data[raw_data["Encoded_SKU_ID"]==sku_id].reset_index(drop=True)
    min_date = sku_1.SALES_DATE.min()
    max_date = sku_1.SALES_DATE.max()
    sku_1_padded = pd.DataFrame({"date":pd.date_range(start=min_date,end=max_date)}).merge(sku_1,left_on="date",right_on="SALES_DATE",how="left")
    sku_1_padded["DAILY_UNITS"] = sku_1_padded["DAILY_UNITS"].fillna(0)
    level_year = sku_1_padded.groupby("year",as_index=False)["DAILY_UNITS"].sum().rename({"DAILY_UNITS":"last_year"},axis=1)
    level_year["year"] = level_year["year"]+1 
    level_year_quarter = sku_1_padded.groupby(["year","quarter"],as_index=False)["DAILY_UNITS"].sum().rename({"DAILY_UNITS":"last_year_same_quarter"},axis=1)
    level_year_quarter["year"] = level_year_quarter["year"]+1
    level_year_month = sku_1_padded.groupby(["year","month"],as_index=False)["DAILY_UNITS"].sum().rename({"DAILY_UNITS":"last_year_same_month"},axis=1)
    level_year_month["year"] = level_year_month["year"]+1
    master_sku_1 = sku_1_padded.merge(level_year,on="year",how="left").merge(level_year_quarter,on=["year","quarter"],how="left").merge(level_year_month,on=["year","month"],how="left")
    #Added own last_year	last_year_same_quarter	last_year_same_month    
    last_quarter = sku_1_padded.groupby(["year","quarter"])["DAILY_UNITS"].sum().shift(1).reset_index().rename({"DAILY_UNITS":"last_quarter"},axis=1)
    last_month = sku_1_padded.groupby(["year","month"])["DAILY_UNITS"].sum().shift(1).reset_index().rename({"DAILY_UNITS":"last_month"},axis=1)
    last_week = sku_1_padded.groupby(["year","week"])["DAILY_UNITS"].sum().shift(1).reset_index().rename({"DAILY_UNITS":"last_week"},axis=1)
    master_sku_1 = master_sku_1.merge(last_quarter,on=["year","quarter"],how="left").merge(last_month,on=["year","month"],how="left").merge(last_week,on=["year","week"],how="left")
    master_sku_1["lag_365_days"] = master_sku_1.set_index("SALES_DATE")["DAILY_UNITS"].shift(365).reset_index()["DAILY_UNITS"]
    master_sku_1["lag_30_days"] = master_sku_1.set_index("SALES_DATE")["DAILY_UNITS"].shift(30).reset_index()["DAILY_UNITS"]
    master_sku_1["lag_7_days"] = master_sku_1.set_index("SALES_DATE")["DAILY_UNITS"].shift(7).reset_index()["DAILY_UNITS"]
    master_sku_1["lag_7_day_Inventory_Fully-Stocked"] = master_sku_1.set_index("SALES_DATE")["Inventory_Fully-Stocked"].shift(7).reset_index()["Inventory_Fully-Stocked"]
    master_sku_1["lag_7_day_Inventory_Moderate"] = master_sku_1.set_index("SALES_DATE")["Inventory_Moderate"].shift(7).reset_index()["Inventory_Moderate"]
    master_sku_1["lag_7_day_Inventory_Out-of-Stock"] =	master_sku_1.set_index("SALES_DATE")["Inventory_Out-of-Stock"].shift(7).reset_index()["Inventory_Out-of-Stock"]	
    
    master_sku_1['PROMO_PRICE'] = master_sku_1['PROMO_PRICE'].replace('?',np.nan)
    master_sku_1['last_promo_price'] = master_sku_1['PROMO_PRICE'].fillna(method='ffill')
    
    # MA Variables
#     master_sku_1["MA_365_days"] = master_sku_1.apply(lambda row : ma_months(row,12),axis=1)
#     master_sku_1["MA_6months"] = master_sku_1.apply(lambda row : ma_months(row,6),axis=1)
#     master_sku_1["MA_2weeks"] = master_sku_1.apply(lambda row : ma_days(row,7),axis=1)
    
    #Added own last_quarter	last_month	last_week
    return master_sku_1


master_data_frame = pd.DataFrame()
for sku in master_sku_list["Encoded_SKU_ID"].values:
    #print(sku)
    master_data_frame = pd.concat([master_data_frame,create_data_for_sku(sku)])
master_data_frame = master_data_frame.fillna(0)


category_name_30_days = master_data_frame.groupby(["CATEGORY_NAME","date"],as_index=False)["lag_30_days"].sum().rename({"lag_30_days":"category_name_lag_30_days"},axis=1)
category_name_7_days = master_data_frame.groupby(["CATEGORY_NAME","date"],as_index=False)["lag_7_days"].sum().rename({"lag_7_days":"category_name_lag_7_days"},axis=1)

ml_name_30_days = master_data_frame.groupby(["ML_NAME","date"],as_index=False)["lag_30_days"].sum().rename({"lag_30_days":"ml_name_lag_30_days"},axis=1)
ml_name_7_days = master_data_frame.groupby(["ML_NAME","date"],as_index=False)["lag_7_days"].sum().rename({"lag_7_days":"ml_name_lag_7_days"},axis=1)

class_name_30_days = master_data_frame.groupby(["CLASS_NAME","date"],as_index=False)["lag_30_days"].sum().rename({"lag_30_days":"class_name_lag_30_days"},axis=1)
class_name_7_days = master_data_frame.groupby(["CLASS_NAME","date"],as_index=False)["lag_7_days"].sum().rename({"lag_7_days":"class_name_lag_7_days"},axis=1)

master_data_frame = master_data_frame.merge(category_name_30_days,on=["CATEGORY_NAME","date"],how="left").merge(category_name_7_days,on=["CATEGORY_NAME","date"],how="left")\
    .merge(ml_name_30_days,on=["ML_NAME","date"],how="left").merge(ml_name_7_days,on=["ML_NAME","date"],how="left")\
        .merge(class_name_30_days,on=["CLASS_NAME","date"],how="left").merge(class_name_7_days,on=["CLASS_NAME","date"],how="left")



## Modeling Using XGBoost

In [ ]:
import xgboost as xg
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',eval_metric="rmsle",\
                  n_estimators = 10, seed = 42)

def perform_forecast_v2(sku_id,master_data_frame,forecast_on = "test"):
    #Added own last_quarter	last_month	last_week
    model_cols = ['month', 'quarter',
       'leap_year', 'year', 'day_of_week', 'week', 'last_year',
       'last_year_same_quarter', 'last_year_same_month', 'last_quarter',
       'last_month', 'last_week', 'lag_30_days', 'lag_7_days',
       'category_name_lag_30_days',	'category_name_lag_7_days',	'ml_name_lag_30_days',
       	'ml_name_lag_7_days',	'class_name_lag_30_days',	'class_name_lag_7_days',
        'Inventory_Fully-Stocked','Inventory_Moderate', 'Inventory_Out-of-Stock',
        'lag_7_day_Inventory_Fully-Stocked', 'lag_7_day_Inventory_Moderate',
       'lag_7_day_Inventory_Out-of-Stock', "consumer_sentiment_index", "gscpi",
        'DAILY_UNITS']
    main_data  = master_data_frame[master_data_frame["Encoded_SKU_ID"]==sku_id]
    model_data = main_data[model_cols]
    model_data["week"] =  model_data["week"].astype(int)
    train_data = model_data[:-7]
    test_data = model_data[-7:]
    X_train = train_data.drop("DAILY_UNITS",axis=1)
    y_train = train_data[["DAILY_UNITS"]]
    X_test = test_data.drop("DAILY_UNITS",axis=1)
    y_test = test_data[["DAILY_UNITS"]]
#    reg = LinearRegression().fit(X_train, y_train)
    reg = xgb_r.fit(X_train,y_train)
    if forecast_on=="test":
        y_pred = reg.predict(X_test)
        master_sku_1_test = main_data[-7:]
        master_sku_1_test["pred"] = y_pred
        master_sku_1_test["sku_id"] = sku_id    
    elif forecast_on=="train":
        y_pred = reg.predict(X_train)
        master_sku_1_test = main_data[:-7]
        master_sku_1_test["pred"] = y_pred
        master_sku_1_test["sku_id"] = sku_id    
    elif forecast_on=="holdout":
        train_data = model_data[:-14]
        test_data = model_data[-14:-7]
        X_train = train_data.drop("DAILY_UNITS",axis=1)
        y_train = train_data[["DAILY_UNITS"]]
        X_test = test_data.drop("DAILY_UNITS",axis=1)
        y_test = test_data[["DAILY_UNITS"]]
        y_pred = reg.predict(X_test)
        master_sku_1_test = main_data[-14:-7]
        master_sku_1_test["pred"] = y_pred
        master_sku_1_test["sku_id"] = sku_id    
    
    return master_sku_1_test[["sku_id","SALES_DATE","DAILY_UNITS","pred"]].reset_index()


# training and predicting on Validation set
all_sku_forecasts_v2 = pd.DataFrame()
for sku in master_sku_list["Encoded_SKU_ID"].values:
    print(sku)
    a = perform_forecast_v2(sku_id=sku,master_data_frame=master_data_frame, forecast_on="test")
    all_sku_forecasts_v2 = pd.concat([all_sku_forecasts_v2,a],axis=0,ignore_index=True)

In [ ]:
# sku level rmse for ensemble decisions
mydf = pd.DataFrame(columns=["sku","rmse"])
for idx,sku in enumerate(master_sku_list["Encoded_SKU_ID"].values):
    # mydict[sku] = mean_squared_error(all_sku_forecasts[all_sku_forecasts["sku_id"]==sku].DAILY_UNITS, all_sku_forecasts[all_sku_forecasts["sku_id"]==sku].pred,squared=False)
    mydf.loc[idx,'sku'] = sku
    mydf.loc[idx,'rmse'] = mean_squared_error(all_sku_forecasts_v2[all_sku_forecasts_v2["sku_id"]==sku].DAILY_UNITS, all_sku_forecasts_v2[all_sku_forecasts_v2["sku_id"]==sku].pred,squared=False)
mydf.to_csv('holdout rmse xgb.csv')


In [ ]:
all_sku_forecasts_v2.to_csv("validation_latest_prediction_xgboost.csv")